In [ ]:
from lxml import html
import requests
import numpy as np
import pandas as pd
import pickle
import osrm

In [ ]:
with open('objects.pickle', 'rb') as f:
    objects = pickle.load(f)

In [ ]:
src = np.array(map(lambda x: (x[1], x[0]), objects['building_apartments']), dtype=float) # типа жилые дома

In [ ]:
process = 0
number_of_apartments = src.shape[0]

for nominator in objects.keys():
    if nominator != 'building_apartments':
        where = list(np.array(map(lambda x: (x[1], x[0]), objects[nominator]), dtype=float))        
        print 'size is %d' % len(where)
        
        res = np.array([])
        
        if len(where) * number_of_apartments < 10000000:
            res = osrm.table(src, where, output='np')[0]
        else :
            res = osrm.table(src[:100], where, output='np')[0]
            k = 100

            chunk = 1
            print '    chunk is %d/210' % chunk
            while k < number_of_apartments:
                part_res = osrm.table(src[k:min(k+100, number_of_apartments)], where, output='np')[0]

                res = np.append(res, part_res, axis=0)
                k += 100
                chunk += 1
                if chunk % 50 == 0:
                    print '    chunk is %d/210' % chunk

        np.save('%s.npy' % nominator, res)
            
        process += 1
        print 'process is %d/39' % process

In [ ]:
# фичи, для которых возможно имеет смысл брать среднее расстояние до 3 ближайших
nearest_3_list = ['shop_convenience', 'amenity_fast_food', 'amenity_cafe', 'highway_bus_stop', \
                  'railway_subway_entrance', 'amenity_school', 'amenity_bank', 'leisure_playground',\
                  'amenity_restaurant', 'shop_supermarket'] 

In [ ]:
final_data = np.ndarray(shape=(20998, 49), dtype=float)
ind = 0
process = 0
index_range = np.array([[i] for i in xrange(20998)])
columns_for_csv = []
for nominator in objects.keys():
    if nominator != 'building_apartments':
        res = np.load('%s.npy' % nominator)
        final_data[:, ind] = res.min(1)
        ind += 1
        columns_for_csv.append(nominator)
        if nominator in nearest_3_list:
            final_data[:, ind] = res[index_range, np.argpartition(res, 3)[:, :3]].mean(1)
            ind += 1
            columns_for_csv.append(nominator + '_3')
            
        process += 1
        print 'process is %d/39' % process

In [ ]:
data = pd.DataFrame(data=final_data, columns=columns_for_csv)

In [ ]:
pd.DataFrame.to_csv(data, 'dataset_without_years.csv',sep='\t')